In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("../../")
from plot_OpSims import plot_OpSims_Nqso_hist_v2

In [ ]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [ ]:
# import convenience functions
import sys 
sys.path.append("../../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

### First, load the ExgalM5 results. 

In [ ]:
NSIDE=64
your_username = "rjassef"
folder_mafoutput = "EM5_{0:d}_v2".format(NSIDE)
resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [ ]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [ ]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

In [ ]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [ ]:
# check keys
dbRuns = list(resultDbs.keys())
bd_keys = list(bundleDicts[dbRuns[0]].keys())
print(bd_keys)

### Now, load the WFD footprint results.

In [ ]:
WFDfp_folder_mafoutput = "WFDfootprint_{0:d}".format(NSIDE)
WFDfp_resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, WFDfp_folder_mafoutput)
WFDfp_metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, WFDfp_folder_mafoutput)

In [ ]:
# get a dictionary of resultDb from given directory
WFDfp_resultDbs = getResultsDbs(WFDfp_resultDbPath)

In [ ]:
# retrieve metricBundles for each opsim run and store them in a dictionary
WFDfp_bundleDicts = dict()
for runName in WFDfp_resultDbs:
    WFDfp_bundleDicts[runName] = bundleDictFromDisk(WFDfp_resultDbs[runName], runName, WFDfp_metricDataPath)

In [ ]:
#Load the WFD footprint for each OpSim run as a mask. 
Key = (1, 'nvisitsLong')
wfd_mask = dict()
for run in WFDfp_resultDbs:
    wfd_footprint = WFDfp_bundleDicts[run][Key].metricValues.filled(0)
    wfd_mask[run] = np.where(wfd_footprint > 750, False, True)

In [ ]:
#Make sure that the plots folder exists.
plots_folder = "plots_all_opsims_WFDfiltered_{0:d}".format(NSIDE)
if not os.path.exists(plots_folder):
    os.mkdir(plots_folder)

### Now make the calculations and plots

In [ ]:
sys.path.append("../../Filter_Curves/")
from lam_eff import lam_eff

In [ ]:
sys.path.append("../")
from Fast_Nqso import Fast_Nqso

In [ ]:
#filters = ["u","g","r","i","z","y"]
filters = ["i"]
mbright = {"u":14.7,
           "g":15.7,
           "r":15.8,
           "i":15.8,
           "z":15.3,
           "y":13.9
          }
Nqso = dict()
Area = dict()
dens = dict()
for filter in filters:
    zmin = 0.3
    zmax = np.min([6.7, (lam_eff['LSST'+filter]/(912.*u.AA) - 1.0).to(1.).value])
    for k, run in enumerate(dbRuns):
        for run_key in bundleDicts[run].keys():
            if run_key[1][-1]==filter:
                Key = run_key
                break
        mask  = wfd_mask[run] | bundleDicts[run][Key].metricValues.mask
        mlim5 = bundleDicts[run][Key].metricValues[~mask]
        filter = Key[1][-1]
        qso_counter = Fast_Nqso("LSST"+filter,"Shen20","A",
                                area=bundleDicts[run][Key].slicer.pixArea*u.sr)
        if filter not in Nqso:
            Nqso[filter] = np.zeros(len(dbRuns))
            Area[filter] = np.zeros(len(dbRuns))
            dens[filter] = np.zeros(len(dbRuns))
        Nqso[filter][k] = np.sum(qso_counter.Nqso(zmin, zmax, mbright[filter], mlim5))
        Area[filter][k] = len(mlim5) * (bundleDicts[run][Key].slicer.pixArea*u.sr).to(u.deg**2).value
        dens[filter][k] = Nqso[filter][k]/Area[filter][k]

In [ ]:
for filter in filters:
    xlabel = r'$N_{\rm QSO}$ expected in $'+filter+r'$-band'
    survey_label = "WFD"
    plot_OpSims_Nqso_hist_v2(filter, Nqso, xlabel=xlabel, 
                             datamin=None, datamax=None, bins=30, figsize=(8,4), 
                             survey_label=survey_label)
    plt.savefig(plots_folder+"/Nqso_{}.png".format(filter))

In [ ]:
for filter in filters:
    xlabel = r'Area expected in $'+filter+r'$-band'
    survey_label = "WFD"
    plot_OpSims_Nqso_hist_v2(filter, Area, xlabel=xlabel,
                             datamin=None, datamax=None, bins=30, figsize=(8,4),
                             survey_label=survey_label)
    plt.savefig(plots_folder+"/Area_{}.png".format(filter))

In [ ]:
for filter in filters:
    xlabel = r'$N_{\rm QSO}/deg^{-2}$ expected in $'+filter+r'$-band'
    survey_label = "WFD"
    plot_OpSims_Nqso_hist_v2(filter, dens, xlabel=xlabel, 
                             datamin=None, datamax=None, bins=30, figsize=(8,4), 
                             survey_label=survey_label)
    plt.savefig(plots_folder+"/qso_density_{}.png".format(filter))

In [ ]:
median_depth = dict()
for filter in filters:
    zmin = 0.3
    zmax = np.min([6.7, (lam_eff['LSST'+filter]/(912.*u.AA)).to(1.).value])
    for k, run in enumerate(dbRuns):
        for run_key in bundleDicts[run].keys():
            if run_key[1][-1]==filter:
                Key = run_key
                break
        mask  = bundleDicts[run][Key].metricValues.mask
        mlim5 = bundleDicts[run][Key].metricValues[~mask]
        filter = Key[1][-1]
        if filter not in median_depth:
            median_depth[filter] = np.zeros(len(dbRuns))
        median_depth[filter][k] = np.median(mlim5)

In [ ]:
for filter in filters:
    xlabel = r'Median depth expected in $'+filter+r'$-band'
    survey_label = "WFD"
    plot_OpSims_Nqso_hist_v2(filter, median_depth, xlabel=xlabel, 
                             datamin=None, datamax=None, bins=30, figsize=(8,4),
                             survey_label=survey_label)
    #plt.savefig(plots_folder+"/qso_density_{}.png".format(filter))